# Final Assignment
## Data Science Tools I
### Professor: Don Dalton

---

### Student: Duncan Ferguson

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Question 1 - Python

### 8 points

By now you should be quite familiar with Pandas' DataFrame object and how to use it. Here you will define an extremely simplified version of this class, designed only to mimic the string output that is given by outputting a Pandas DataFrame or a column from a Pandas DataFrame.

Define a class called SimpleDataFrame. The constructor (`__init__`) should expect the data to be given as a list of lists and the columns as a list of strings. Here is an example instantiation:

```python
df = SimpleDataFrame(
  data=[[5, 2, 3, 6, 9], [1, 8, 2, 6, 3], [1, 9, 2, 2, 4]], 
  columns=["col1", "col2", "col3"]
)
```

The data and columns should be stored as instance variables. The approach with which you choose to represent the data and columns is up to you, but you should consider how you may want to store this information and any additional information in order to complete the following two methods.

Define the `__str__(self)` method that returns an string representation of the data frame. This should look similar to the output provided by Pandas' DataFrame object, with an RangeIndex as the left-most column and with the data arranged as columns below its corresponding column names. Here is an example output from running `print(df)` given the `df` variable above:

```
  col1  col2  col3
0 5     1     1
1 2     8     9
2 3     2     2
3 6     6     2
4 9     3     4
```

Your output does not need to match this perfectly, but should be similar and readable.

Define a `__getitem__(self, key)` method for this class. `__getitem__` is a dunder/magic method that allows you to use the `[]` syntax on an instance of the class. For a data frame, we would normally expect `df['col1']`, if `df` was a Pandas DataFrame, to return a Pandas Series object representing that column. Since we are only mimicking the output and not also defining Series, `__getitem__` should only return a string, much like `__str__`, but such that only the given column is represented. For example, `print(df['col2'])` should output something like:

```
col2
1
8
2
6
3
```

Run the provided code cell with the example code to test your class.


In [179]:
class SimpleDataFrame:
  def __init__(self, data, columns):
    self.data = data
    self.columns = columns
    self.flat_list = sum(self.data, [])  # Flattening out the list of lists
    self.new_list = [self.flat_list[i::len(self.data[0])] for i in range(len(self.data[0]))]  # Turning the individual columns into rows


  def __str__(self):
    self.format_columns = "{:>15}" * (len(self.columns) + 1)
    self.str_list = str(self.format_columns.format("", *self.columns)) + "\n"
    self.str_list2 = ""
    for row in enumerate(self.new_list):
      new_row = row[1]
      new_row.insert(0,row[0])
      self.str_list2 = self.str_list2 + str(self.format_columns.format(*new_row)) + "\n"
    return self.str_list + self.str_list2

  def __getitem__(self, key):
    """Uses dunder/magic method that allows you to use [] syntax on an instance of the class"""
    self.new_data = []
    self.new_cols = []

    for col in enumerate(self.columns):
      if col[1] in key:
        self.new_data.append(self.data[col[0]])
        self.new_cols.append(self.columns[col[0]])

    # Flattening and reshaping
    self.flat_list = sum(self.new_data, [])  # Flattening out the list of lists
    self.new_list2 = [self.flat_list[i::len(self.new_data[0])] for i in range(len(self.new_data[0]))]  #
    self.format_columns2 = "{:>15}" * (len(self.new_cols) + 1)

    self.str_list22 = ""
    for row in enumerate(self.new_list2):
      new_row2 = row[1]
      new_row2.insert(0,row[0])
      self.str_list22 = self.str_list22 + str(self.format_columns2.format(*new_row2)) + "\n"


    return str(self.format_columns2.format("", *self.new_cols)) + "\n" + self.str_list22


In [183]:
# Run this to test your SimpleDataFrame class
df = SimpleDataFrame(data=[[5, 2, 3, 6, 9], [1, 8, 2, 6, 3], [1, 9, 2, 2, 4]], columns=["col1", "col2", "col3"])

print(df)
print()
print(df["col2"])

                          col1           col2           col3
              0              5              1              1
              1              2              8              9
              2              3              2              2
              3              6              6              2
              4              9              3              4


                          col2
              0              1
              1              8
              2              2
              3              6
              4              3



# Question 2 - Algorithm Analysis

It is typical in a coding interview to be asked to write a method that computes some output based on some input, often a string or array. It is also typical to be asked to analyze the runtime (and space) complexity of your answer. The question you will answer is as follows:
> Given an array of integer values and a target value $q$, write a method that determines if there exists a pair of elements in the array that sum to $q$.

For example, this method would return `True` for the array `[5, 2, 7, 8, 3, 4]` and `q=11` since 7 and 4, both present in the array, sum to 11. Moreover, 8 and 3 also sum to 11. It does not matter what two values are part of the sum, only that such a pair exists. Given the same array but with `q=4`, the method would return `False` because although 4 (the value of $q$) and 2 (adding 2 to itself sums to $q$) are in the array, no two distinct values sum to 4.

Your task for parts (a) and (b) is to define two different answers to this question, then analyze their runtime. Below is a code cell defining a method called `test_method` that will run a handful of test cases for the methods you define. Go ahead and run this code cell so you'll have the method available to you for testing.

In [363]:
def test_method(method):
  nums = [7, -4, 60, 17, 31, 100, -50]

  # Test normal cases
  assert method(nums, 117)
  assert method(nums, -19)
  assert not method(nums, 14)
  assert not method(nums, 200)

  # Test edge cases
  assert not method([], 42)
  assert not method([50], 50)
  assert not method([50], 100)
  assert method([2, 2], 4)

  print("Tests passed successfully")

**For parts (a) and (b), define the method based on the given instructions, then analyze the runtime of your implementation in a docstring comment just below the header of the method.** Upon calling `test_method` with your method, you will get an AssertionError if you failed any test, or otherwise get a confirmation message if you passed all tests.

## Part (a)

### 6 points

Define a method called `method1` that solves this question in a brute-force manner. Iterate over all possible pairs of elements in the given array, each time checking if their sum is equal to $q$. If any pair's sum is $q$, return True. If no pairs sum to $q$, return False.

Test your method using the provided `test_method`.

Analyze the runtime of `method1` in the docstring using Big-Oh notation. Provide a brief explanation for why the runtime is what you state.

In [367]:
def method1(nums, q):
  """This is a brute force method for checking if two numbers add up the q"""
  # if len(nums) < 2:
  #   return False
  # elif len(nums) == 2 and sum(nums) == q:
  #   return True

  # Creating Combos
  combo = [(nums[i],nums[j]) for i in range(len(nums)) for j in range(i+1, len(nums))]
  for bo in combo:
      if sum(bo) == q:
          return True
  return False

test_method(method1)

Tests passed successfully


### WRITE the Answer here Touch this up
Time Complexity O(n^2)
space complexity O(1)


## Part (b)

### 6 points

Define a method called `method2` that solves this question using sorting. Start by sorting the array first. Create two indexes called `left` and `right`. `left` will start at 0 and `right` at the last index of the array. (For this to work, you cannot be fancy with Python's syntax and use `-1` for `right`; use the *positive integer* that is the last index.) So long as `left` is strictly less than `right`, check the following:
* If the elements at `left` and `right` sum to `q`, return True.
* If their sum is less than `q`, *increment* `left` by 1.
* If their sum is greater than `q`, *decrement* `right` by 1.

The big picture is that `left` and `right` move inward toward the middle of the array and take advantage of the fact that the array is sorted in order to find a pair of elements that sum to `q`.

Test your method using the provided `test_method`.

Analyze the runtime of `method2` in the docstring using Big-Oh notation. Provide a brief explanation for why the runtime is what you state. Assume that the sorting methods provided by Python run in O(n log n) time.

In [381]:
def method2(numbers, q):
    numbers.sort()
    left, right = 0, len(numbers)-1
    while left < right:
        answer = numbers[left] + numbers[right]
        if answer == q:
            return True
        elif answer < q:
            left +=1
        elif answer > q:
            right -= 1
    return False
test_method(method2)

Tests passed successfully


### WRITE the Answer here
Time Complexity: O(n). The input array is traversed at most once. Since the sort complexit it O(n log(n)) Then O(n) + O(n log(n)) = O(n log(n))
Space Complexity: O(1). The only extra space that is used is to store the answer(sum) and two other indexes

# Question 3 - Data Manipulation

The WTA dataset used in this question is also used for Question 4.

## Part (a)

### 3 points

Import the two .csv files on matches from the Women's Tennis Association tour in 2019 and 2020, either from the following URLs or by downloading the files from Canvas and loading them from your local repository.
* https://raw.githubusercontent.com/JeffSackmann/tennis_wta/master/wta_matches_2019.csv
* https://raw.githubusercontent.com/JeffSackmann/tennis_wta/master/wta_matches_2020.csv

Combine these two datasets into one by stacking them together, placing the 2020 data below the 2019 data. Ensure that the index values for the 2020 are adjusted to continue off of the 2019 index.

There are many columns that we can ignore for these questions. Keep just the following columns: `'tourney_name', 'surface', 'match_num', 'winner_name', 'winner_rank', 'winner_ht', 'w_1stIn', 'loser_name', 'loser_rank', 'loser_ht', 'l_1stIn'`. Print the **shape** and **tail** of the combined dataset.


In [404]:
wta_2019 = pd.read_csv('wta_matches_2019.csv')
# wta_2019.sort_values(by=['tourney_date'], inplace=True, ascending=True)
wta_2019.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2019-0300,Luxembourg,Hard,32,I,20191014,270,201504,2.0,NaN,...,73.0,38.0,16.0,14.0,17.0,24.0,26.0,1840.0,74.0,803.0
1,2019-0300,Luxembourg,Hard,32,I,20191014,271,201514,NaN,NaN,...,30.0,20.0,8.0,9.0,2.0,6.0,76.0,793.0,122.0,538.0
2,2019-0300,Luxembourg,Hard,32,I,20191014,272,201697,NaN,NaN,...,50.0,35.0,8.0,10.0,7.0,10.0,70.0,858.0,104.0,600.0
3,2019-0300,Luxembourg,Hard,32,I,20191014,273,201620,NaN,NaN,...,25.0,8.0,6.0,7.0,4.0,10.0,84.0,730.0,65.0,944.0
4,2019-0300,Luxembourg,Hard,32,I,20191014,274,214981,3.0,NaN,...,38.0,28.0,11.0,10.0,3.0,6.0,43.0,1306.0,150.0,396.0


In [405]:
wta_2020 = pd.read_csv('wta_matches_2020.csv')
# wta_2020.sort_values(by=['tourney_date'], inplace=True, ascending=True)
wta_2020.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2020-1049,Auckland,Hard,32,I,20200106,300,200033,1,NaN,...,50.0,31.0,12.0,9.0,12.0,15.0,10.0,3935.0,82.0,743.0
1,2020-1049,Auckland,Hard,32,I,20200106,299,200033,1,NaN,...,22.0,12.0,2.0,7.0,0.0,5.0,10.0,3935.0,25.0,1734.0
2,2020-1049,Auckland,Hard,32,I,20200106,298,202468,NaN,NaN,...,56.0,34.0,9.0,13.0,7.0,13.0,82.0,743.0,39.0,1353.0
3,2020-1049,Auckland,Hard,32,I,20200106,297,200033,1,NaN,...,45.0,26.0,5.0,9.0,1.0,4.0,10.0,3935.0,73.0,805.0
4,2020-1049,Auckland,Hard,32,I,20200106,296,216153,3,NaN,...,62.0,35.0,8.0,13.0,11.0,18.0,25.0,1734.0,262.0,223.0


In [406]:
wta = pd.concat([wta_2019, wta_2020], ignore_index=True)
cut_wta = wta[['tourney_name', 'surface', 'match_num', 'winner_name', 'winner_rank', 'winner_ht', 'w_1stIn', 'loser_name', 'loser_rank', 'loser_ht', 'l_1stIn']].copy()
print("Shape", np.shape(cut_wta))
cut_wta.tail()

Shape (4097, 11)


,tourney_name,surface,match_num,winner_name,winner_rank,winner_ht,w_1stIn,loser_name,loser_rank,loser_ht,l_1stIn
4092,Fed Cup QLS R1: SUI vs CAN,Hard,4,Jil Belen Teichmann,68.0,170.0,39.0,Gabriela Dabrowski,448.0,NaN,50.0
4093,Fed Cup QLS R1: USA vs LAT,Hard,1,Sofia Kenin,7.0,170.0,33.0,Anastasija Sevastova,41.0,169.0,40.0
4094,Fed Cup QLS R1: USA vs LAT,Hard,2,Serena Williams,9.0,175.0,39.0,Jelena Ostapenko,40.0,177.0,48.0
4095,Fed Cup QLS R1: USA vs LAT,Hard,3,Jelena Ostapenko,40.0,177.0,49.0,Sofia Kenin,7.0,170.0,47.0
4096,Fed Cup QLS R1: USA vs LAT,Hard,4,Anastasija Sevastova,41.0,169.0,75.0,Serena Williams,9.0,175.0,60.0


## Part (b)
### 2 points

Suppose we want to get information on the final match of Wimbledon in 2019 (the 2020 event was cancelled due to the pandemic). Extract the rows that contain the matches from the `'Wimbledon'` tournament, then from those extract the final match. Note that `match_num`, as noted by the author of this dataset, is a value that is inconsistent across tournaments, sometimes starting from 1 and going up, other times counting down. Knowing that Serena Williams is perhaps the greatest tennis player to ever live (moreover one of the greatest athletes), check the data for yourself to determine if you should use the minimum `match_num` or maximum. (Serena did end up losing this match, but she's still one of the greats.)

You must programmatically extract the row in question. In other words, you will not receive credit if you find the correct row and hard-code your answer.

In [ ]:
# YOUR CODE HERE

## Part (c)

### 4 points

A quick scan of the data should show you that many of the players' heights are missing from the dataset. To get an idea of how much of this data is missing, first determine how many *unique* players are represented in this dataset by combining the players under the columns `'winner_name'` and `'loser_name'`. (There is a particular operation used to combine these columns. If you don't already know the right method to call, search the internet for an answer or ask your peers/instructor.)

Then, calculate for how many unique players the corresponding height column is missing. Height is given by `'winner_ht'` and `'loser_ht'`, both in centimeters.

Compute the ratio of players with missing heights over total number of players to get a percentage of players with currently unknown (i.e. unrecorded) heights.

In [ ]:
# YOUR CODE HERE

## Part (d)

### 4 points

Given that the player ranks are provided in this dataset, we might be curious to see what a typical match-up is like regarding ranking. Are players of similar rank often paired against each other? Or is player rank perhaps not necessarily used to determine who is matched with whom?

To briefly study these questions, plot a scatterplot of `'loser_rank'` versus `'winner_rank'`. To make the plot a little more interesting, change the color of the data points depending on the `'surface'` being played on.

Using in-line comments in your code cell or as a separate text cell, **provide at least three statements** regarding the plot you created, noting any trends, patterns, outliers, things that may require more exploration, statistical questions that could be interesting to answer, or anything else that stands out to you. You might consider enlarging the plot to get a better look.

In [ ]:
# YOUR CODE HERE

# Question 4 - Hypothesis Testing

Using the WTA dataset from the previous questions, we will perform a hypothesis test on one aspect of the data. 

In case you are unfamiliar with the gameplay rules of tennis, each player alternates serving the ball to start each rally. Players get two shots to make the service shot into the correct box on the opponent's side of the court, the service box diagonally opposite from the server. As such, players typically hit the first serve pretty hard (for female tennis players, commonly around 100mph-115mph at the professional level) since they have a second chance if they miss.

Suppose you speculate that players that end up winning their matches likely have more serves that make it in on the first attempt. After all, a harder serve is harder to return and gives the serving player better control of the rally. To answer this, you propose a t-test to determine if there is a *significant difference between the means* of the sampling distributions of number of first serves in for winners versus that of losers. These values correspond to the `'w_1stIn'` and `'l_1stIn'` columns.

## Part (a)

### 2 points

To start, state the null and alternative hypotheses either as commented code or in a text cell below using the conventional notation.

YOUR ANSWER HERE

## Part (b)
### 2 points

Plot distribution plots of the `'w_1stIn'` and `'l_1stIn'` columns. Along with each plot, print the **mean** and **skew** of each column. Include an in-line comment stating if the plot is approximately normal or not.

In [ ]:
# YOUR CODE HERE

## Part (c)
### 2 points

Perform a Levene's Test on these two columns to determine if the variance of each are roughly equal or not. Note that each column has missing values, so you should drop those values in order to get actual results from the test. Your output should provide an alpha value (assumed to be 0.05), a p-value, a test statistic, and statement that the variance is either close to equal or not.

In [ ]:
# YOUR CODE HERE

## Part (d)
### 3 points

Perform the t-test for two independent samples on these two columns. Based on the resulting p-value and an alpha of 0.05, state if the null hypothesis is retained (failed to be rejected) or rejected. Provide an interpretation of the result either as an in-line comment or as a separate text cell.

In [ ]:
# YOUR CODE HERE

# Question 5 - Time Series Analysis

Background info on this dataset: I (Terron) have been studying Japanese basically since my first year as an undergraduate. Despite my Japanese heritage, my family does not speak the language, so I grew up only knowing English (and some Spanish from school). I have studied Japanese in university, but I have mostly studied in my free time. One of the many tools I have used for learning Japanese is a site called [WaniKani](https://www.wanikani.com/), made by the company [Tofugu](https://www.tofugu.com/) which posts and creates many useful articles and resources about learning Japanese. WaniKani is a flashcard-type system that presents kanji, vocabulary, and radicals in a pre-determined order, starting with easy characters and moving up to harder ones. Each time you learn a new item, a "review" item is generated to test you in increasingly longer intervals.

As it turns out, WaniKani has an API through which you can programmatically access data about your progress. For this question, I wrote a script to query the API and download a few months' worth of information on my progress from the date I started using the service in 2018. To keep the data simple, I consolidated the dates to get per-day data, with the data of interest being the number of reviews generated for that particular day (in other words, how many new words I learned that day). This is the time series used for this question.

## Part (a)
### 3 points

Download the `JapaneseReviews.csv` file from Canvas and place it in your local repository. Import the file into a Pandas DataFrame. Convert the `'date'` column to datetime objects, then set the index to be the `'date'` column and in doing so convert the DataFrame into a Series. Plot the data to observe the time series.

In [ ]:
# YOUR CODE HERE

## Part (b)
### 2 points

Run the Augmented Dickey-Fuller test on the data to determine if the time series is already stationary. You may use the `run_adftest` method provided in lecture.

If the time series is not stationary, state so in an in-line comment, then perform a first-order difference and re-run the test.

In [ ]:
# YOUR CODE HERE

## Part (c)
### 3 points

Plot the auto-correlation function (ACF) to get an indication of what type of ARIMA model may apply to this dataset. Refer to the table below to determine which model is best.

Provide your model choice either in a text cell below or as comments in the code cell alongside your code for the ACF plot. 

Shape | Indicated Model
--- | ---
Exponential, decay to zero | AR - Use PACF plot to identify order
Alternating positive and negative, decaying to zero | AR - Use PACF plot to identify order
One or more spikes, rest are essentially zero  | MA - Order identified by where plot becomes zero
Decay, starting after a few lags | ARMA
All zero or close to zero | Data is essentially random
High values at fixed intervals | Include seasonal autoregressive term / difference
No decay to zero | The series is not stationary

In [ ]:
# YOUR CODE HERE

## Part (d)
### 3 points

If you chose a model in part (c) that calls for use of ARIMA, use the ADF test, ACF plot, and PACF plots to determine the values of p, d, and q. Train an ARIMA model on these parameters and plot the results as shown in lecture.

If you determined in part (c) that the data is essentially random, then calculate and output the mean of the time series. (The best prediction that can be made from a time series with random values is its mean. There is no point in attempting to create a model for random data.)

In [ ]:
# YOUR CODE HERE

# Question 6 - Machine Learning

This question utilizes the [Mushroom Data Set](https://archive.ics.uci.edu/ml/datasets/Mushroom) provided by University of California, Irvine. 

The dataset contains a bunch of features regarding a sample of mushrooms labeled as either `p` for poisonous or `e` for edible. The objective is to design a classifier that will discern between poisonous and edible mushrooms.

## Part (a)
### 3 points

Download the `mushrooms.csv` file from Canvas and place it in your local repository. Import the data into a Pandas DataFrame. 

In order for the `score` mechanism to work properly, the `'class'` labels must be integer values. Replace `p` with the value 1 and `e` with the value 0. This means we are viewing `p` (poisonous) as the positive label and `e` (edible) as the negative label.

Print the head of the dataset to confirm the data has been imported and the `'class'` column has been edited appropriately.

In [ ]:
# YOUR CODE HERE

## Part (b)
### 3 points

It should be clear that every column in this dataset is categorical. In order for us to use these features in machine learning models, we must convert them to continuous features. Here we do so using one-hot encodings.

Separate the dataset in the feature matrix X (all columns except `'class'`) and target vector y (just the column `'class'`). Use `pd.get_dummies` to convert X into a matrix of one-hot encodings instead of categorical features.

After extracting X and y, use `train_test_split` to split the data into training and test sets. Since the data is split randomly, to keep your results consistent, set `random_state=0` when calling `train_test_split`.

In [ ]:
# YOUR CODE HERE

## Part (c)
### 2 points

Let's see how well a simple classifier can do on this one-hot encoded data set. Instantiate a `GaussianNB` classifier, fit it on the training set, and output the `score` based on the classifier's predictions on the test set. 

You may find that you have to reshape the predictions vector using `reshape(-1,1)` in order to compute the score.

In [ ]:
# YOUR CODE HERE

## Part (d)
### 3 points

One possible reason why the score is not particularly good is because there are so many features, particularly due to the one-hot encoding. Use the `SelectPercentile` class to get the top 30% of features based on statistical significance. Ensure that the resulting training set has notably fewer features by printing the shape of the original training set X and the transformed training set X.

In addition, output the features that `SelectPercentile` ended up choosing as the most important features.

In [ ]:
# YOUR CODE HERE

## Part (e)
### 2 points

With most of the features discarded, train a new `GaussianNB` classifier based on the selected features in the training set. Transform the test set to also contain only the selected features, then output the resulting score. You should find that the score is not better than the classifier that was trained on all features.

In [ ]:
# YOUR CODE HERE

## Part (f)
### 2 points

To provide a more robust evaluation of this model, we should use cross-validation. Since this classifier is "naive" (meaning it is a relatively simple model), it should perform better if provided more of the overall data as training. As such, it is probably better to use 10-fold cross-validation.

Perform a stratified 10-fold cross-validation of the new classifier that is based on the selected features. Output the 10 scores produced by the cross-validation along with the mean of the 10 scores.

In [ ]:
# YOUR CODE HERE

## Part (g)
### 4 points

Assuming 1 ("poisonous") is our positive label and 0 ("edible") is our negative label, consider the two types of errors that can occur with this binary classification. What would the consequences be of a false positive? How about false negative? Answer these questions in an in-line comment or as a separate text cell.

Based on your answer, determine if precision, recall, or f1-score would be the best metric to base our model on. Compute and output the chosen metric score.

In [ ]:
# YOUR CODE HERE

# Question 7 - Reflection

### 5 points

This course, Data Science Tools I, has covered a number of different topics that have hopefully enlightened you to what you can possibly do with data science. It is my (Terron's) belief that tools of such power and significance should be wielded and used for the greater good. Yes, data scientists can make a lot of money and being financially stable is a key facet of living a comfortable lifestyle. But ultimately to me, the greatest fulfillment I get from my job and my past occupations is the positive impact I can have on others and on society.

How do you think you will utilize the tools you have learned? What problems exist in the world that you would like answers to? Are there any populations of interest to you that could be better understood or better represented publicly? What concerns you about how statistics are presented or could be used for or against certain people? Why study data science? What careers might you consider going into that utilize these tools?

You do not have to answer all of these questions. I am mostly looking for a general reflection from you on data science as a whole. I know we've only scratched the surface of many of these subjects and there is plenty more to learn, but I would like to get your overall thoughts on the course, what you've learned, what you can do with what you've learned, etc.

While you are welcome to answer this directly in a paragraph or two, you are encouraged to be creative in how you answer. You could write a haiku, a limerick, draw a comic or a picture, whatever medium best allows you to express yourself. If you need to attach another file to your submission as a result, zip the extra file(s) with your notebook and submit the zipped files to Canvas.


YOUR ANSWER HERE OR IN A SEPARATE FILE ZIPPED WITH YOUR NOTEBOOK